In [ ]:
import os
import sys
import torch
agent_position_data = torch.load("./agent_position_data.pt")
agent_heading_data = torch.load("./agent_heading_data.pt")
agent_velocity_data = torch.load("./agent_velocity_data.pt")
agent_shape_data = torch.load("./agent_shape_data.pt")
agent_category_data = torch.load("./agent_category_data.pt")
agent_mask_data = torch.load("./agent_mask_data.pt")

map_point_position_data = torch.load("./map_point_position_data.pt")
map_point_vector_data = torch.load("./map_point_vector_data.pt")
map_point_orientation_data = torch.load("./map_point_orientation_data.pt")
map_polygon_center_data = torch.load("./map_polygon_center_data.pt")
map_polygon_type_data = torch.load("./map_polygon_type_data.pt")
map_polygon_on_route_data = torch.load("./map_polygon_on_route_data.pt")
map_polygon_tl_status_data = torch.load("./map_polygon_tl_status_data.pt")
map_polygon_has_speed_limit_data = torch.load("./map_polygon_has_speed_limit_data.pt")
map_polygon_speed_limit_data = torch.load("./map_polygon_speed_limit_data.pt")
map_mask_data = torch.load("./map_mask_data.pt")

reference_line_position_data = torch.load("./reference_line_position_data.pt")
reference_line_vector_data = torch.load("./reference_line_vector_data.pt")
reference_line_orientation_data = torch.load("./reference_line_orientation_data.pt")
reference_line_valid_mask_data = torch.load("./reference_line_valid_mask_data.pt")

static_objects_position_data = torch.load("./static_objects_position_data.pt")
static_objects_heading_data = torch.load("./static_objects_heading_data.pt")
static_objects_shape_data = torch.load("./static_objects_shape_data.pt")
static_objects_category_data = torch.load("./static_objects_category_data.pt")
static_objects_valid_mask_data = torch.load("./static_objects_valid_mask_data.pt")

current_state_data = torch.load("./current_state_data.pt")

candidate_trajectories_data = torch.load("./candidate_trajectories_data.pt")

probability_data = torch.load("./probability_data.pt")


In [ ]:


ckpt_path = "/home/bydguikong/nuplan/exp/exp/training/scope/2025.04.01.13.59.13/checkpoints/last.ckpt"
# ckpt_path = "/home/bydguikong/yy_ws/PlanScope/checkpoints/last.ckpt"
checkpoint = torch.load(ckpt_path,map_location="cuda")
# 配置 planner 相关参数
sys.path.append('/home/bydguikong/yy_ws/PlanScope/')
from src.planners.ml_planner_utils import global_trajectory_to_states, load_checkpoint
from nuplan.planning.training.modeling.torch_module_wrapper import TorchModuleWrapper
from src.models.pluto.scope_model import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

planner = PlanningModel(
    dim =  128,
    state_channel =  6,
    polygon_channel =  6,
    history_channel =  9,
    history_steps =  21,
    future_steps =  80,
    encoder_depth =  4,
    decoder_depth =  4,
    drop_path =  0.2,
    dropout =  0.1,
    num_heads =  4,             #
    num_modes =  12,            #
    use_ego_history =  False,
    state_attn_encoder =  True,
    state_dropout =  0.75,
    use_hidden_proj = True,     #
    cat_x = True,               #非常重要
    ref_free_traj =  True,     #
    recursive_decoder =  False,
    residual_decoder =  False,
    multihead_decoder =  False,
    independent_detokenizer =  False,
    mvn_loss =  False,
    wtd_with_history =  False,
    plot_debug = True,
)
# print("planner = ",planner)
mk, uk = planner.load_state_dict(load_checkpoint(ckpt_path), strict=False)
planner.to(device)
planner.eval()  # 设置为评估模式



In [ ]:
import plotly.graph_objects as go
import numpy as np
from plotly.colors import qualitative
# 定义地图车道颜色组，每组包含中心线、左边界和右边界的颜色
color_pairs = [
    {"center": "#2C3E50", "left": "#1f77b4", "right": "#d62728"},  # 组0：中心线带蓝色调
    {"center": "#34495E", "left": "#17becf", "right": "#ff7f0e"},  # 组1：中心线带蓝灰调
    {"center": "#8E44AD", "left": "#0074D9", "right": "#FF4136"},  # 组2：中心线带紫色调
    {"center": "#16A085", "left": "#2E86C1", "right": "#E74C3C"},  # 组3：中心线带青绿色调
    {"center": "#27AE60", "left": "#2980B9", "right": "#E67E22"},  # 组4：中心线带绿色调
    {"center": "#D35400", "left": "#3498DB", "right": "#F39C12"},  # 组5：中心线带橙色调
    {"center": "#C0392B", "left": "#5DADE2", "right": "#E74C3C"},  # 组6：中心线带红色调
]

# 计算所有帧中 agent 数量的最大值
max_A = max([agent_position_item.shape[0] for agent_position_item in agent_position_data[0]])

# 使用 Plotly 提供的定性配色方案作为 agent 的颜色池（不足则循环使用）
agent_colors_pool = qualitative.Plotly  
agent_colors = [agent_colors_pool[i % len(agent_colors_pool)] for i in range(max_A)]

frames = []
n_frames = len(agent_position_data)
print(f"共{n_frames}帧")
step = n_frames//5
print(f"step = {step}")
for frame_idx, (
    agent_position,
    agent_heading,
    agent_velocity,
    agent_shape,
    agent_category,
    agent_valid_mask,
    map_point_position,
    map_point_vector,
    map_point_orientation,
    map_polygon_center,
    map_polygon_type,
    map_polygon_on_route,
    map_polygon_tl_status,
    map_polygon_has_speed_limit,
    map_polygon_speed_limit,
    map_valid_mask,
    reference_line_position,
    reference_line_vector,
    reference_line_orientation,
    reference_line_valid_mask,
    static_objects_position,
    static_objects_heading,
    static_objects_shape,
    static_objects_category,
    static_objects_valid_mask,
    current_state,
    candidate_trajectories,
    probability
) in enumerate(zip(
    agent_position_data,
    agent_heading_data,
    agent_velocity_data,
    agent_shape_data,
    agent_category_data,
    agent_mask_data,
    map_point_position_data,
    map_point_vector_data,
    map_point_orientation_data,
    map_polygon_center_data,
    map_polygon_type_data,
    map_polygon_on_route_data,
    map_polygon_tl_status_data,
    map_polygon_has_speed_limit_data,
    map_polygon_speed_limit_data,
    map_mask_data,
    reference_line_position_data,
    reference_line_vector_data,
    reference_line_orientation_data,
    reference_line_valid_mask_data,
    static_objects_position_data,
    static_objects_heading_data,
    static_objects_shape_data,
    static_objects_category_data,
    static_objects_valid_mask_data,
    current_state_data,
    candidate_trajectories_data,
    probability_data
)):
    if frame_idx % step != 0:
        continue
    dummy_input = [
        agent_position.to(device),
        agent_heading.to(device),
        agent_velocity.to(device),
        agent_shape.to(device),
        agent_category.to(device),
        agent_valid_mask.to(device),
        map_point_position.to(device),
        map_point_vector.to(device),
        map_point_orientation.to(device),
        map_polygon_center.to(device),
        map_polygon_type.to(device),
        map_polygon_on_route.to(device),
        map_polygon_tl_status.to(device),
        map_polygon_has_speed_limit.to(device),
        map_polygon_speed_limit.to(device),
        map_valid_mask.to(device),
        reference_line_position.to(device),
        reference_line_vector.to(device),
        reference_line_orientation.to(device),
        reference_line_valid_mask.to(device),
        static_objects_position.to(device),
        static_objects_heading.to(device),
        static_objects_shape.to(device),
        static_objects_category.to(device),
        static_objects_valid_mask.to(device),
        current_state.to(device)
    ]
    # if frame_idx < select or frame_idx >= select+5:
    #     continue
    print_structure(dummy_input)
    output = planner.forward(dummy_input)
    # print(type(output))
    # print("candidate_trajectories.shape",output["candidate_trajectories"].shape)
    # print("probability.shape",output["probability"].shape)
    map_item = map_point_position[0]      # shape: (D1, D2, P, 2)
    map_valid_item = map_valid_mask[0]            # shape: (D1, P)
    traces = []
    D1, D2, P, _ = map_item.shape
    bs, num_agents, num_history, _ = agent_position.shape
    print(f"agent_position.shape = {agent_position.shape}")
    # 遍历每个车道
    for lane_idx in range(D1):
        color_set = color_pairs[lane_idx % len(color_pairs)]
        # 取出当前车道的有效掩码，形状为 (P,)
        lane_mask = map_valid_item[lane_idx]  # 布尔张量
        
        # 取出当前车道的各条边界数据，shape: (D2, P, 2)
        lines = map_item[lane_idx]
        # 假设第0条边界为中心线，第1条为左边界，第2条为右边界
        if D2 < 3:
            continue  # 如果当前车道数据不足3条，跳过
        center_line = lines[0]
        left_bound   = lines[1]
        right_bound  = lines[2]
        
        # 根据掩码选择有效的点
        center_valid = center_line[lane_mask]  # shape: (num_valid, 2)
        left_valid   = left_bound[lane_mask]
        right_valid  = right_bound[lane_mask]
        
        # 添加中心线
        traces.append(go.Scatter(
            x=center_valid[:, 0].cpu().numpy().tolist() if center_valid.numel() > 0 else [],
            y=center_valid[:, 1].cpu().numpy().tolist() if center_valid.numel() > 0 else [],
            mode='lines',
            name=f"Lane {num_agents+lane_idx} Center",
            # line=dict(color=color_set["center"], dash="dot")
            line=dict(color="gray" ,dash="dot"),
            showlegend=True,
        ))
        # 添加左边界
        traces.append(go.Scatter(
            x=left_valid[:, 0].cpu().numpy().tolist() if left_valid.numel() > 0 else [],
            y=left_valid[:, 1].cpu().numpy().tolist() if left_valid.numel() > 0 else [],
            mode='lines',
            name=None,
            # name=f"Lane {lane_idx} Left",
            # line=dict(color=color_set["left"])
            line=dict(color="gray"),
            showlegend=False,
        ))
        # 添加右边界
        traces.append(go.Scatter(
            x=right_valid[:, 0].cpu().numpy().tolist() if right_valid.numel() > 0 else [],
            y=right_valid[:, 1].cpu().numpy().tolist() if right_valid.numel() > 0 else [],
            mode='lines',
            name=None,
            # name=f"Lane {lane_idx} Right",
            # line=dict(color=color_set["right"])
            line=dict(color="gray"),
            showlegend=False,
        ))
    # print(f"agent_position.shape = {agent_position.shape}")
    # print(f"agent_heading.shape = {agent_heading.shape}")
    # print(f"agent_shape.shape = {agent_shape.shape}")
    # print(f"agent_valid_mask.shape = {agent_valid_mask.shape}")
    # 绘制智能体轨迹和安全盒
    # 数据格式说明：agent_position.shape = [bs, A, L, 2]
    #           agent_heading.shape  = [bs, A, L]
    #           agent_shape.shape    = [bs, A, L, 2] (长和宽)
    #           agent_valid_mask.shape = [bs, A, L]
    
    for agent_idx in range(num_agents):
        # 取出单个智能体的历史数据（假设 batch 0）
        pos = agent_position[0, agent_idx]         # (L, 2)
        heading = agent_heading[0, agent_idx]        # (L,)
        shape_info = agent_shape[0, agent_idx]         # (L, 2)
        valid_mask = agent_valid_mask[0, agent_idx]    # (L,)
        
        # 过滤有效的轨迹点
        valid_mask_bool = valid_mask.bool()
        valid_positions = pos[valid_mask_bool]        # (num_valid, 2)
        # print("agent_colors.size = ",len(agent_colors))
        # print("agent_idx = ",agent_idx)
        if valid_positions.numel() > 0:
            traces.append(go.Scatter(
                x = valid_positions[:,0].cpu().numpy().tolist(),
                y = valid_positions[:,1].cpu().numpy().tolist(),
                # mode='lines+markers',
                mode='lines',
                # name=f"Agent {agent_idx} Trajectory",
                name=None,
                # line=dict(dash="solid", color=agent_colors[agent_idx])
                line=dict(dash="solid",color = "yellow"),
                showlegend=False,
            ))
            
        # 绘制最后一帧的安全盒，仅当最后一帧有效时绘制
        if valid_mask[-1]:
            last_pos = pos[-1]       # (2,)
            last_heading = heading[-1].item()  # 标量
            # print("last_heading = ",last_heading)
            last_shape = shape_info[-1]         # (2,) [length, width]
            b_box = last_shape[0].item()
            l_box = last_shape[1].item()
            # 定义局部坐标系下的矩形角点（顺时针顺序）
            local_corners = np.array([
                [ l_box/2,  b_box/2],
                [ l_box/2, -b_box/2],
                [-l_box/2, -b_box/2],
                [-l_box/2,  b_box/2],
                [ l_box/2,  b_box/2]  # 闭合多边形
            ])
            # 构造旋转矩阵（注意：朝向角的定义依据实际情况，假设以弧度为单位）
            R = np.array([
                [np.cos(last_heading), -np.sin(last_heading)],
                [np.sin(last_heading),  np.cos(last_heading)]
            ])
            rotated_corners = local_corners.dot(R.T)
            # 将角点平移到最后的位置
            corners = rotated_corners + last_pos.cpu().numpy()
            traces.append(go.Scatter(
                x=corners[:,0].tolist(),
                y=corners[:,1].tolist(),
                mode='lines',
                name=f"Agent {agent_idx} Safety Box",
                line=dict(color="black", width=2),
                showlegend=True,
            ))
    # 规划轨迹可视化
    # candidate_trajectories: [bs, R, M, L, 3]   (x, y, yaw)
    # probability: [bs, R, M]
    
    candidate_trajs = output["candidate_trajectories"]
    prob = output["probability"]
    
    # candidate_trajs = candidate_trajectories
    # prob = probability
    
    # 转换为 numpy 数组（假设 batch size 为1）
    candidate_trajs_np = candidate_trajs[0].cpu().detach().numpy()  # shape: [R, M, L, 3]
    prob_np = prob[0].cpu().detach().numpy()                          # shape: [R, M]
    
    R, M, L, F = candidate_trajs_np.shape
    # 找出所有参考车道中概率最高的轨迹对应的 (r, m) 索引（整体最优）
    overall_best_idx = np.unravel_index(np.argmax(prob_np), (R, M))
    
    # 为每个参考车道单独找出最佳候选轨迹
    best_m_per_r = [np.argmax(prob_np[r]) for r in range(R)]
    
    # 为不同的参考车道分配不同颜色（使用 Plotly 定性配色方案）
    ref_colors = qualitative.Plotly
    ref_colors = [ref_colors[i % len(ref_colors)] for i in range(R)]
    # 用于图例控制：每个参考车道只显示一次图例
    ref_legend_added = [False] * R
    
    for r in range(R):
        for m in range(M):
            traj = candidate_trajs_np[r, m, :, :]   # shape: (L, 3)
            x_traj = traj[:, 0]
            y_traj = traj[:, 1]
            # 默认候选轨迹使用参考车道对应的颜色
            line_color = ref_colors[r]
            line_width = 0.6
            # 如果当前候选轨迹是该参考车道下的最优轨迹，则加宽显示
            if m == best_m_per_r[r]:
                line_width = 2
            # 如果当前候选轨迹同时为整体最优轨迹，额外标记末端
            if (r, m) == overall_best_idx:
                line_color = ref_colors[r]
                line_width = 2
                overall_best = True
            else:
                overall_best = False

            
            # 绘制候选轨迹
            traces.append(go.Scatter(
                x=x_traj.tolist(),
                y=y_traj.tolist(),
                mode='lines',
                name=None,
                line=dict(color=line_color, width=line_width),
                opacity=0.6,
                showlegend=False
            ))
            # 如果当前候选轨迹是整体最优轨迹，则在末端添加点标记
            if overall_best:
                traces.append(go.Scatter(
                    x=[x_traj[-1]],
                    y=[y_traj[-1]],
                    mode='markers',
                    name="Overall Best Endpoint",
                    marker=dict(color=ref_colors[r], size=10),
                    showlegend=False,
                ))
                
    # motion = output["output_prediction"]
    # print("motion.shape = ",motion.shape)
    # motion_pred = motion[0].cpu().detach().numpy()
    # A, L, F = motion_pred.shape
    # for i in range(A):
    #     traj = motion_pred[i]  # shape: (L, 5)
    #     # 对每个轨迹，将连续的点对分段绘制，颜色由白色到蓝色渐变
    #     for j in range(L-1):
    #         # 计算颜色渐变因子：0时为白色，1时为蓝色
    #         factor = j / (L - 1)
    #         # 白色：RGB (255,255,255)，蓝色：RGB (0,0,255)
    #         r = 0
    #         g = int(round(255 * factor))
    #         b = int(round(255 * (1 - factor)))
    #         color_hex = f"#{r:02x}{g:02x}{b:02x}"
    #         # 分段坐标：当前点和下一点
    #         x_segment = [traj[j, 0], traj[j+1, 0]]
    #         y_segment = [traj[j, 1], traj[j+1, 1]]
    #         # 对于每个智能体的第一段，添加图例项
    #         if j == 0:
    #             seg_name = f"Motion Prediction Agent {i}"
    #         else:
    #             seg_name = None
    #         traces.append(go.Scatter(
    #             x=x_segment,
    #             y=y_segment,
    #             mode='lines',
    #             name=seg_name,
    #             line=dict(color=color_hex, width=0.6),
    #             showlegend=(seg_name is not None)
    #         ))

    # 绘制参考车道线，参考车道数据维度为 [bs, R, L, 2]
    # print("reference_line_position.shape = ",reference_line_position.shape)
    # print("reference_line_orientation.shape = ",reference_line_orientation.shape)
    # print("reference_line_valid_mask.shape = ",reference_line_valid_mask.shape)
    # print("reference_line_vector.shape = ",reference_line_vector.shape)
    ref_line_np = reference_line_position[0].cpu().detach().numpy()  # shape: [R, L, 2]
    for r in range(R):
        ref_line = ref_line_np[r]  # shape: [L, 2]
        x_ref = ref_line[:, 0]
        y_ref = ref_line[:, 1]
        traces.append(go.Scatter(
            x=x_ref.tolist(),
            y=y_ref.tolist(),
            mode='lines',
            name=f"Reference Line {r}",
            line=dict(color=ref_colors[r], width=3, dash="dash"),
            showlegend=True,
        ))
        
    # 创建 Plotly 图形，将所有车道线拼接显示
    fig = go.Figure(data=traces)
    fig.update_layout(
        title="Map Lanes, Agent Trajectories & Planning Paths",
        xaxis_title="X",
        yaxis_title="Y",
        yaxis=dict(scaleanchor="x", scaleratio=1),
        width=700,
        height=500,
    )
    fig.show()
    
    # break